In [1]:
## Definition of the DrugSDA SCP server, including basic operations such as connect, disconnect, list_tools, and parse_result.
import asyncio
import json
from mcp.client.streamable_http import streamablehttp_client
from mcp import ClientSession

DrugSDA_Tool_SERVER_URL = "https://scp.intern-ai.org.cn/api/v1/mcp/2/DrugSDA-Tool"      ## DrugSDA-Tool Server

class DrugSDAClient:    
    def __init__(self, server_url: str):
        self.server_url = server_url
        self.session = None
        
    async def connect(self):
        print(f"server url: {self.server_url}")
        try:
            self.transport = streamablehttp_client(
                url=self.server_url,
                headers={"SCP-HUB-API-KEY": "sk-a0033dde-b3cd-413b-adbe-980bc78d6126"}
            )
            self.read, self.write, self.get_session_id = await self.transport.__aenter__()
            
            self.session_ctx = ClientSession(self.read, self.write)
            self.session = await self.session_ctx.__aenter__()

            await self.session.initialize()
            session_id = self.get_session_id()
            
            print(f"✓ connect success")
            return True
            
        except Exception as e:
            print(f"✗ connect failure: {e}")
            import traceback
            traceback.print_exc()
            return False
    
    async def disconnect(self):
        try:
            if self.session:
                await self.session_ctx.__aexit__(None, None, None)
            if hasattr(self, 'transport'):
                await self.transport.__aexit__(None, None, None)
            print("✓ already disconnect")
        except Exception as e:
            print(f"✗ disconnect error: {e}")
    
    async def list_tools(self):        
        try:
            tools_list = await self.session.list_tools()
            print(f"tool count: {len(tools_list.tools)}")
            
            for i, tool in enumerate(tools_list.tools, 1):
                print(f"{i:2d}. {tool.name}")
                if tool.description:
                    desc_line = tool.description.split('\n')[0]
                    print(f"    {desc_line}")
            
            print(f"✓ Get tool list success")
            return tools_list.tools
            
        except Exception as e:
            print(f"✗ Get tool list fail: {e}")
            return []
    
    def parse_result(self, result):
        try:
            if hasattr(result, 'content') and result.content:
                content = result.content[0]
                if hasattr(content, 'text'):
                    return json.loads(content.text)
            return str(result)
        except Exception as e:
            return {"error": f"parse error: {e}", "raw": str(result)}

In [3]:
'''
Example 1: Virtual Screening of Drug-like Molecules.
(a) Given a list of candidate molecules in SMILES format (smiles_list), the drug-likeness (QED) and toxicity (LD50) scores are first calculated. Molecules that clearly fail to meet predefined thresholds are filtered out. 
(b) The remaining candidates then undergo docking against a target protein—a multi-step process involving protein structure retrieval, structural preprocessing, binding pocket identification, file format conversion, and finally, quick molecular docking. 
(c) Compounds with a binding affinity lower than −7.0 kcal/mol are selected as high-potential candidates.
'''
async def main():
    client = DrugSDAClient(DrugSDA_Tool_SERVER_URL)
    if not await client.connect():
        print("connection failed")
        return
    
    smiles_list = ['O=C(Nc1cccc2c1CCCC2)N1CCc2c([nH]c3ccccc23)C1c1cccc(F)c1F',
        'O=C(Nc1cccc(C2CN3CCSC3c3ccccc32)c1)Nc1ccc2ccccc2c1',
        'Cc1ccccc1N1CCN(C2=Nc3cc(Cl)ccc3Nc3ccc(F)cc32)CC1',
        'O=C1NCc2ccc(cc2)Cc2cccc(c2)CNC(=O)c2coc(n2)-c2cccc(c2)-c2cccc(c2)-c2cccc(c2)-c2cccc(c2)COc2cccc1c2',
        'O=C(Nc1ccc2c(c1)C(F)(F)C(F)(F)CC2)N1CCc2ccccc2C1c1c[nH]c2ccccc12',
        'O=C(c1cccc(F)c1F)N1CCc2c(c3ccccc3n2Cc2cccc(C=Cc3ccc4cc(F)ccc4n3)c2)C1',
        'O=C(CN(C(=O)c1cccc(F)c1)N1C(=O)C2C3c4ccccc4C(c4ccccc43)C2C1=O)c1ccccc1',
        'O=C(Nc1cccc2c1CCCC2)N1CCc2c([nH]c3ccccc23)C1c1ccc(F)cc1',
        'O=C(c1ccc2c(c1)N(Cc1ccccc1)C(=O)c1ccccc1S2(=O)=O)c1ccc2ccccc2c1',
        'O=C(Nc1cccc(-c2cccc(C(F)(F)F)c2)c1)c1cccc(-c2ccc3ccccc3c2)c1',
        'O=C(Nc1ccc2c(c1)C(F)(F)C(F)(F)CC2)N1CCc2ccccc2C1C1CCCCC1',
        'O=C(c1ccccc1F)N1CCN2C(=O)c3ccccc3C12c1ccc(Cl)cc1',
        'CCc1ccc(C2c3[nH]c4ccccc4c3CCN2C(=O)CN2C(=O)c3ccccc3C2=O)cc1',
        'O=C(Nc1ccc(F)c(C(F)(F)F)c1)Nc1cccc2ccc(Nc3cccc(-c4ccccc4)c3)nc12',
        'O=C(Nc1cccc(Oc2ccccc2)c1)Nc1cccc(-c2cccc(-c3nc4cc(C(F)(F)F)ccc4[nH]3)c2)c1',
        'O=C(c1cccc(-c2cccc(C(F)(F)F)c2)c1)N1CCc2c(-c3cccc(F)c3)ccnc2C1',
        'O=C(c1ccc2c(c1)C(F)(F)C(c1ccccc1)C2c1ccc(Cl)cc1)N1CCOCC1',
        'O=C(Nc1ccc2c(c1)C(F)(F)C(F)(F)CC2)N1CCc2ccccc2C1c1ccc(F)cc1F',
        'O=C(C1CC1c1cccc(F)c1)N1CCN(C2c3ccccc3-c3ccccc32)CC1',
        'CNc1ccc2c(c1)NC1(CCN(C(=O)c3ccc4[nH]ccc4c3)CC1)c1ccccc1C2',
        'O=C1c2cccc3cccc(c23)C1c1cccc(Oc2ccccc2)c1',
        'O=C(c1ccccc1)N1CCN(C(=O)c2cc(Cc3n[nH]c(=O)c4ccccc34)ccc2F)CC1',
        'CC(=O)Nc1cccc(Nc2c3ccccc3nc3ccccc23)c1',
        'O=C(Nc1cccc(-c2cccc(-c3nc4cc(C(F)(F)F)ccc4[nH]3)c2)c1)Nc1cccc2c1CCCC2',
        'O=C(Nc1ccc2c(c1)N(S(=O)(=O)c1cccs1)CCC2)N1CC2CC(C1)c1cccc(=O)n1C2',
        'O=C(NC1CCCC1)C1Cc2ccccc2CN1C(=O)c1cccc2ccccc12',
        'O=C(Nc1ccccc1)c1cccc(Nc2ccc3c(c2)OCc2ccccc2C3=O)c1',
        'Cc1cc(C)cc(NC(=O)N2CCc3c([nH]c4ccccc34)C2c2ccccc2F)c1',
        'O=C(Nc1ccc2c(c1)C(F)(F)C(F)(F)CC2)c1ccc2c(c1)C(=O)N(Cc1ccccc1)C2=O',
        'O=C(c1cccc(-c2cccc(CN3CCN(c4ccccc4)CC3)c2)c1)N1CCC(O)CC1',
        'CC1(C)N=C(N)N=C(N)N1c1ccc(C(=O)C(c2ccc(N3C(N)=NC(N)=NC3(C)C)cc2)c2ccc(C(F)(F)F)cc2)cc1',
        'O=C1c2ccccc2C(=O)c2c1ccc1c2C(c2ccccc2)(c2ccccc2)OC1',
        'CC(C)(C)c1ccc(C(=O)Nc2cccc(Nc3ccc4c(c3)OCc3ccccc3C4=O)c2)cc1',
        'O=C(c1cccc(Cc2nnc(O)c3c2CCCC3)c1)N1CCN(C(=O)c2ccco2)CC1',
        'O=C1CC(c2ccccc2)c2c(c3ccccc3c3ccccc23)N1',
        'Cc1cccc(C2C3=C(COC3=O)Nc3ccc4ccccc4c32)c1',
        'Cc1ccc2c(c1)C(=Nc1cccc(-c3ccc(N4CCOCC4)cc3)c1)c1ccccc1-2',
        'O=C(c1cccc(Cc2ccc(F)cc2)c1)N1CCN(C(=O)c2ccc3ccccc3c2)CC1',
        'COc1cccc(C23c4ccccc4C(=O)N2CCN3C(=O)c2cc(F)c(F)c(F)c2)c1',
        'O=C(c1ccc(F)cc1F)N1CCN2C(=O)c3ccccc3C12c1ccc(Cl)cc1',
        'O=C(c1cccc(F)c1)N1CCN2C(=O)c3ccccc3C12c1ccc(Cl)cc1',
        'O=C1C2Cc3c([nH]c4ccccc34)C(c3ccc4c(c3)OCO4)N2C(=O)CN1c1ccc(F)cc1',
        'O=C(Nc1cccc(-c2cccc(-c3nc4cc(C(F)(F)F)ccc4[nH]3)c2)c1)Nc1ccc2c(c1)CCC2',
        'O=C(Nc1cccc(Oc2ccc3ccccc3c2)c1)c1cccc(-c2ccc3ccccc3c2)c1',
        'O=C(c1ccc(-c2cccc(F)c2)cc1)N1CCc2c([nH]c3ccccc23)C1c1ccc2c(c1)OCO2',
        'O=C(Nc1cccc2c1CCCC2)N1CCc2c([nH]c3ccccc23)C1c1ccc(Cl)cc1',
        'O=C(Nc1ccc2c(c1)C(F)(F)C(F)(F)CC2)N1CCc2ccccc2C1c1ccc(C(F)(F)F)cc1',
        'O=C(Nc1cccc(-c2cccc(-c3nc4cc(C(F)(F)F)ccc4[nH]3)c2)c1)Nc1cccc2ccccc12',
        'O=C(Nc1cccc(F)c1)N1CCc2c([nH]c3ccccc23)C1c1ccccc1F',
        'O=C(NCc1cccc(-c2cccc(-c3cc4c[nH]ccc-4n3)c2O)c1)Nc1ccc(F)cc1']

    ## step 1. calculate QED scores, call SCP tool calculate_mol_drug_chemistry
    result = await client.session.call_tool(
        "calculate_mol_drug_chemistry",
        arguments={
            "smiles_list": smiles_list
        }
    )
    result_data = client.parse_result(result)
    QED_result = result_data["metrics"]
    print ("Compute QED score finish ...")
    
    ## step 2. Calculate LD50 scores, call SCP tool pred_molecule_admet
    result = await client.session.call_tool(
        "pred_mol_admet",
        arguments={
            "smiles_list": smiles_list,
            "smiles_file": ""
        }
    )
    result_data = client.parse_result(result)
    admet_result = result_data["json_content"]
    print ("Predict admet finish ...")
    
    ## step 3. Filter molecules with QED ≥ 0.6 and LD50_Zhu ≥ 3.0
    select_smiles_list = []
    for i in range(len(smiles_list)):
        smiles = smiles_list[i]
        QED = QED_result[i]["qed"]
        LD50 = admet_result[i]["admet_predictions"]["LD50_Zhu"]
        if QED >= 0.6 and LD50 >= 3.0:
            select_smiles_list.append(smiles)
            
    print (len(select_smiles_list), select_smiles_list[0])
    
    ## step 4. Retrieve and download the target protein structure, call SCP tool retrieve_protein_data_by_pdbcode.
    pdb_code = "6vkv"
    result = await client.session.call_tool(
        "retrieve_protein_data_by_pdbcode",
        arguments={
            "pdb_code": pdb_code
        }
    )

    result_data = client.parse_result(result)
    pdb_path = result_data["pdb_path"]
    print ("download protein structure: ", pdb_path)
    
    ## step 5. Extract main chain, call SCP tool save_main_chain_pdb
    result = await client.session.call_tool(
        "save_main_chain_pdb",
        arguments={
            "pdb_file_path": pdb_path,
            "main_chain_id": ""
        }
    )
    
    result_data = client.parse_result(result)
    pdb_path = result_data["out_file"]
    print ("extract protein chain: ", pdb_path)
    
    ## step 6. Fix PDB, call SCP tool fix_pdb_dock
    result = await client.session.call_tool(
        "fix_pdb_dock",
        arguments={
            "pdb_file_path": pdb_path
        }
    )
    
    result_data = client.parse_result(result)
    pdb_path = result_data["fix_pdb_file_path"]
    print ("fix protein pdb: ", pdb_path)
    
    ## step 7. Identify binding pockets, call SCP tool run_fpocket
    result = await client.session.call_tool(
        "run_fpocket",
        arguments={
            "pdb_file_path": pdb_path
        }
    )
    
    result_data = client.parse_result(result)
    best_pocket = result_data["pockets"][0]
    print ('pocket info: ', best_pocket)
    
    ## step 8. Convert SMILES to PDBQT format, call SCP tool convert_smiles_to_format
    result = await client.session.call_tool(
        "convert_smiles_to_format",
        arguments={
            "inputs": select_smiles_list,
            "target_format": "pdbqt"
        }
    )
    result_data = client.parse_result(result)
    ligand_paths = [x["output_file"] for x in result_data["convert_results"]]
    print (ligand_paths)
    
    ## step 9. Convert receptor PDB to PDBQT format, call SCP tool convert_pdb_to_pdbqt_dock
    result = await client.session.call_tool(
        "convert_pdb_to_pdbqt_dock",
        arguments={
            "pdb_file_path": pdb_path
        }
    )
    result_data = client.parse_result(result)
    receptor_path = result_data["output_file"]
    print (receptor_path)
    
    ## step 10. Molecular docking, call SCP tool quick_molecule_docking
    result = await client.session.call_tool(
        "quick_molecule_docking",
        arguments={
            "receptor_path": receptor_path,
            "ligand_paths": ligand_paths,
            "center_x": best_pocket["center_x"],
            "center_y": best_pocket["center_y"],
            "center_z": best_pocket["center_z"],
            "size_x": best_pocket["size_x"],
            "size_y": best_pocket["size_y"],
            "size_z": best_pocket["size_z"]
        }
    )
    result_data = client.parse_result(result)
    print ("docking finish ...")
    
    ## step 11. Select final candidate molecules with binding affinity ≤ −7.0 kcal/mol
    final_smiles_list = []
    index = 0
    for item in result_data["docking_results"]:
        affinity = item['affinity']
        if affinity <= -7.0:
            final_smiles_list.append(select_smiles_list[index])
        index += 1
    
    print (final_smiles_list)
    
    await client.disconnect()
    

if __name__ == '__main__':
    await main()

server url: https://scp.intern-ai.org.cn/api/v1/mcp/2/DrugSDA-Tool
✓ connect success
Compute QED score finish ...
Predict admet finish ...
6 O=C(c1ccccc1F)N1CCN2C(=O)c3ccccc3C12c1ccc(Cl)cc1
download protein structure:  /root/lwj/wll/code/DrugAgentTools/exp_data/6vkv.pdb
extract protein chain:  /root/lwj/wll/code/DrugAgentTools/exp_data/6vkv_chainA.pdb
fix protein pdb:  /root/lwj/wll/code/DrugAgentTools/exp_data/6vkv_chainA_fix_0227_103920365.pdb
pocket info:  {'pocket_id': 1, 'score': 0.373, 'center_x': 2.0607, 'center_y': -59.641, 'center_z': -4.5997, 'size_x': 30.6014, 'size_y': 40.0, 'size_z': 34.2147, 'n_atoms': 47}
['/root/lwj/wll/code/DrugAgentTools/exp_data/smiles_0_0227_103922666.pdbqt', '/root/lwj/wll/code/DrugAgentTools/exp_data/smiles_1_0227_103922666.pdbqt', '/root/lwj/wll/code/DrugAgentTools/exp_data/smiles_2_0227_103922666.pdbqt', '/root/lwj/wll/code/DrugAgentTools/exp_data/smiles_3_0227_103922666.pdbqt', '/root/lwj/wll/code/DrugAgentTools/exp_data/smiles_4_0227_103922666